# Taller Text Mining

In [1]:
%%configure -f
{ "conf":{
"spark.pyspark.python": "python3",
"spark.pyspark.virtualenv.enabled": "true",
"spark.pyspark.virtualenv.type":"native",
"spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
}}

In [2]:
import nltk
#from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1574276126980_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True
[nltk_data] Downloading package stopwords to
[nltk_data]     /var/lib/livy/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /var/lib/livy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!

In [38]:
sc.install_pypi_package('pandas')
#sc.install_pypi_package('nltk')
sc.install_pypi_package('tqdm')
sc.install_pypi_package('Ipython')
sc.install_pypi_package('tabulate')
sc.install_pypi_package('boto3')
sc.install_pypi_package('scipy')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package already installed for current Spark context!
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1110, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [5]:
import sys, os, csv, glob, json, uuid, pickle, math
import nltk
import numpy as np, scipy, pandas as pd
from operator import itemgetter
from IPython.display import HTML, display
import tabulate
from tqdm import tqdm
from pyspark.ml.feature import Word2Vec, Word2VecModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
bucketName = 'taller4'
URI = sc._gateway.jvm.java.net.URI
Path = sc._gateway.jvm.org.apache.hadoop.fs.Path
FileSystem = sc._gateway.jvm.org.apache.hadoop.fs.FileSystem
fs = FileSystem.get(URI("s3://"+bucketName), sc._jsc.hadoopConfiguration())

# We can now use the Hadoop FileSystem API (https://hadoop.apache.org/docs/current/api/org/apache/hadoop/fs/FileSystem.html)
#fs.listStatus(Path('/user/njaram15/'))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
CONTENT_INDEX = 9

csv.field_size_limit(sys.maxsize)
CONTENT_PATH = '/tallerTextMining/inputs/contents/'
TOKENS_PATH = '/tallerTextMining/inputs/tokens/'
CENTROIDS_PATH = '/tallerTextMining/inputs/centroids/'
MODEL_PATH = '/tallerTextMining/model/'

#fs.mkdirs(Path(CONTENT_PATH))
#fs.mkdirs(Path(TOKENS_PATH))
#fs.mkdirs(Path(CENTROIDS_PATH))
#fs.mkdirs(Path(MODEL_PATH))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df = spark.read.csv('s3://'+bucketName+'/tallerTextMining/inputs/*.csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
len(df.rdd.collect())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

999

In [10]:
count = 0

for line in df.rdd.collect():
    count = count + 1
    content = line['content']
    cname = CONTENT_PATH + str(count) + '.txt'
    tname = TOKENS_PATH + str(count) + '.tokens'
    #write_s3file(cname,content)
    if not fs.exists(Path('s3://'+bucketName+cname)):
        sc.parallelize([content]).saveAsTextFile('s3://'+bucketName+cname)    
    sentences = ""
    for sentence in nltk.sent_tokenize(content):
        sentences = sentences + sentence.lower() + "\n"

    #write_s3file(tname, sentences)
    if not fs.exists(Path('s3://'+bucketName+tname)):
        sc.parallelize([sentences]).saveAsTextFile('s3://'+bucketName+tname)    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
from nltk.stem.porter import *
from nltk.corpus import stopwords
def cleanStopWordAndStemming(sentences):
    stop_words = stopwords.words('english')
    stemmer = PorterStemmer()
    sen = []
    for sentence in sentences:
        senten = [word for word in sentence if word not in stop_words]
        singles = [stemmer.stem(token) for token in senten]
        sen.append(singles)
    return sen

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
sentences = []
df2 = spark.read.text('s3://'+bucketName+'/tallerTextMining/inputs/tokens/*.tokens')
for line in df2.collect():
    sentences.append(nltk.word_tokenize(line['value'].strip()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
sentences = cleanStopWordAndStemming(sentences)
print(sentences[0])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['joachim', 'neander', 'calvinist', 'theologian', 'often', 'hike', 'valley', 'outsid', 'düsseldorf', ',', 'germani', ',', 'write', 'hymn', '.']

In [18]:
inputs = [(x,) for x in sentences]
doc = spark.createDataFrame(inputs, ["sentence"])
word2Vec = Word2Vec(vectorSize=5, seed=42, inputCol="sentence", outputCol="model")
model = word2Vec.fit(doc)
model.getVectors().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+
|       word|              vector|
+-----------+--------------------+
|    rahmani|[-0.0112890955060...|
|      brink|[0.11128488183021...|
|   promenad|[0.09507359564304...|
|    acronym|[0.04120612144470...|
|  forgotten|[0.13175040483474...|
|     justif|[-0.2294008284807...|
|     teresa|[-0.0794001594185...|
|      lover|[0.09871610999107...|
|     comedi|[0.13516199588775...|
|  regularli|[0.18409946560859...|
|      fanci|[0.06045890972018...|
|       elit|[0.39351123571395...|
|    speaker|[-0.2253546267747...|
|       chee|[0.12783752381801...|
|       lion|[0.06552257388830...|
|       rate|[0.63716894388198...|
|     pepper|[0.09586166590452...|
|       2014|[0.15399555861949...|
|uncertainti|[0.15482957661151...|
|nonetheless|[0.03948763385415...|
+-----------+--------------------+
only showing top 20 rows

In [19]:
if not fs.exists(Path('s3://'+bucketName+'/tallerTextMining/model/word2vec-model')):
    model.save('s3://'+bucketName+'/tallerTextMining/model/word2vec-model')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
model = None
model = Word2VecModel.load('s3://'+bucketName+'/tallerTextMining/model/word2vec-model')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
words = model.getVectors().select('word').collect()
vocab = dict([(v.word, k) for k, v in enumerate(words)])
#write_s3file('taller2/model/w2v-lc-vocab.json', json.dumps(vocab))
if not fs.exists(Path('s3://'+bucketName+'/tallerTextMining/model/w2v-lc-vocab.json')):
    sc.parallelize([json.dumps(vocab)]).saveAsTextFile('s3://'+bucketName+'/tallerTextMining/model/w2v-lc-vocab.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
from pyspark.sql.functions import format_number as fmt

vectors = model.getVectors()
t = vectors.filter(vectors.word == 'mother').select('vector').collect()[0]['vector']
# como senate y alabama no están en el dataset de prueba se simulan con ceros
s = np.array([0.0,0.0,0.0,0.0,0.0])
a = np.array([0.0,0.0,0.0,0.0,0.0])
r = t+s-a
# r = a-t+s
model.findSynonyms(r, 10).select("word", fmt("similarity", 5).alias("similarity")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------+
|        word|similarity|
+------------+----------+
|      mother|   1.00000|
|          tv|   0.99624|
|      jacket|   0.99155|
|        movi|   0.99134|
|       whole|   0.98891|
|      sitcom|   0.98794|
|       baton|   0.98666|
|        song|   0.98525|
|      runway|   0.98351|
|totalitarian|   0.98344|
+------------+----------+

In [24]:

from  pyspark.sql.functions import input_file_name
df3 = spark.read.text('s3://'+bucketName+'/tallerTextMining/inputs/contents/*.txt')
df3 = df3.withColumn("filename", input_file_name())
df3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|               value|            filename|
+--------------------+--------------------+
|Joachim Neander w...|s3://taller4/tall...|
|WASHINGTON  —   W...|s3://taller4/tall...|
|When Indira Islas...|s3://taller4/tall...|
|On the morning of...|s3://taller4/tall...|
|In the fall of 20...|s3://taller4/tall...|
|Hours before the ...|s3://taller4/tall...|
|President Obama d...|s3://taller4/tall...|
|One night six yea...|s3://taller4/tall...|
|After the bullet ...|s3://taller4/tall...|
|The Season 7 “Rea...|s3://taller4/tall...|
|On the night of N...|s3://taller4/tall...|
|WASHINGTON  —   O...|s3://taller4/tall...|
|Canada, our No. 1...|s3://taller4/tall...|
|A   of   lead exp...|s3://taller4/tall...|
|Updated: 11:50 p....|s3://taller4/tall...|
|• Hundreds of tho...|s3://taller4/tall...|
|Thousands of year...|s3://taller4/tall...|
|GLEN ELDER, Kan. ...|s3://taller4/tall...|
|Danny Cahill stoo...|s3://taller4/tall...|
|They called him t...|s3://talle

In [26]:
import time

t0 = time.time()
dic = {k:v for k,v in vectors.collect()}
for fname in df3.rdd.collect():
    rdd = sc.parallelize([fname['value']])
    centroid_in = list(rdd.map(lambda x: [dic[w] if w in dic.keys() else [0.0,0.0,0.0,0.0,0.0] for w in x.lower().split(' ')]).map(lambda x: np.mean(x, axis=0)).collect()[0])
    out_dict = { fname['filename'] : centroid_in}
    json_file = '/inputs/centroids/' + fname['filename'].split('/')[-2].replace('.txt', '.json')
    if not fs.exists(Path('s3://'+bucketName+'/tallerTextMining'+json_file)):
        sc.parallelize([json.dumps(out_dict)]).saveAsTextFile('s3://'+bucketName+'/tallerTextMining/'+json_file)
    #write_s3file(json_file, json.dumps(out_dict))
print('time elapsed: ', time.time()-t0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

time elapsed:  907.1658599376678

In [27]:
sents = [(s,) for s in sentences]
df4 = spark.createDataFrame(sents)
dictionary = df4.rdd.map(lambda x: x[0]).reduce(lambda x,y: list(np.unique(list(x)+list(y))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
centroid_dict = {}
dfCentroide = spark.read.text('s3://'+bucketName+'/tallerTextMining/inputs/centroids/*.json')
for fname in dfCentroide.rdd.collect():
    d = json.loads(fname['value'])
    centroid_dict.update(d)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
clean_centroid_dict = {k: centroid_dict[k] for k in centroid_dict if not np.isnan(centroid_dict[k][0]).any()}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
def get_embedding(x, out=False):
    if x in dictionary:
        return vectors.filter(vectors.word == x).select('vector').collect()[0]['vector']
    else:
        return np.zeros(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
from scipy import spatial
def score_document(q_embeddings, d_centroid):
    individual_csims = [(1 - spatial.distance.cosine(qin, d_centroid)) for qin in q_embeddings]
    return (sum(individual_csims)/len(q_embeddings))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
documents = []
df5 = spark.read.text('s3://'+bucketName+'/tallerTextMining//inputs/contents/*.txt')
for fname in df5.rdd.collect():
    documents.append(nltk.word_tokenize(fname['value'].strip()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
query = 'mother buys meat'
query_words = nltk.word_tokenize(query.lower())
query_ins = [get_embedding(x) for x in query_words]
q_len = len(query_ins)
print('Num words in query: ', len(query_words), 'Num query word in vectors: ', q_len)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Num words in query:  3 Num query word in vectors:  3

In [40]:
scores_in_in = []
for k,v in clean_centroid_dict.items():
    scores_in_in.append((k, score_document(query_ins, v[0])))

scores_in_in = sorted(scores_in_in, key=itemgetter(1), reverse=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/tmp/1574283488638-0/local/lib64/python3.6/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)

In [41]:
print('TOP 5 IN-IN:')
top_5_in_in = [x[0] for x in scores_in_in[:5]]

for fname in top_5_in_in:
    print(fname.split('/')[-2])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TOP 5 IN-IN:
772.txt
267.txt
545.txt
926.txt
936.txt